In [1]:
import pandas as pd
pd.options.display.max_colwidth = 1000
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20.2f}'.format)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
from IPython.display import clear_output,display

In [2]:
# Load environment variables from .env file
from dotenv import load_dotenv

# Load .env file
load_dotenv()
import random

import os
from vexa_client import VexaClient

import os
TOKEN = os.getenv('ADMIN_API_TOKEN')
url = "http://localhost:18056"
admin_client = VexaClient(
    base_url=url,
    admin_key=TOKEN  # From docker-compose.yml
)

import time

In [4]:
new_user = admin_client.create_user(email=f"{random.randint(1, 1000000)}@example.com", name="test")



DEBUG: Making POST request to http://localhost:18056/admin/users
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: {'email': '113796@example.com', 'name': 'test'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Sat, 07 Jun 2025 22:00:31 GMT, Sat, 07 Jun 2025 22:00:31 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '134', 'content-type': 'application/json'}
DEBUG: Response content: {"email":"113796@example.com","name":"test","image_url":null,"max_concurrent_bots":1,"id":4,"created_at":"2025-06-07T22:00:31.941243"}...


In [5]:

user_id = new_user['id']
user_id
token_info = admin_client.create_token(user_id=user_id)
user_api_key = token_info['token']
client = VexaClient(
    base_url=url,
    api_key=user_api_key
)


DEBUG: Making POST request to http://localhost:18056/admin/users/4/tokens
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Sat, 07 Jun 2025 22:00:36 GMT, Sat, 07 Jun 2025 22:00:36 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '113', 'content-type': 'application/json'}
DEBUG: Response content: {"user_id":4,"id":4,"token":"9JT4p4Ms2WqGc951ZJtyu7oOltStI9onzWdFkEPC","created_at":"2025-06-07T22:00:37.655315"}...


In [6]:

meeting_url="https://meet.google.com/xdy-txjo-fyh"
native_meeting_id = meeting_url.split("/")[-1]  # "xyz-abcd-123"
meeting_info = client.request_bot(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
    bot_name="Vexa",
  #  language='es',
    task = 'transcribe'
)
meeting_id = meeting_info['id']


DEBUG: Making POST request to http://localhost:18056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': '9JT4p4Ms2WqGc951ZJtyu7oOltStI9onzWdFkEPC'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'xdy-txjo-fyh', 'bot_name': 'Vexa', 'task': 'transcribe'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Sat, 07 Jun 2025 22:00:43 GMT, Sat, 07 Jun 2025 22:00:43 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '401', 'content-type': 'application/json'}
DEBUG: Response content: {"id":4,"user_id":4,"platform":"google_meet","native_meeting_id":"xdy-txjo-fyh","constructed_meeting_url":"https://meet.google.com/xdy-txjo-fyh","status":"active","bot_container_id":"a06675e931d224b995711174211ab7e1e27a670ba0ee67c3b707dada795c98f5","start_time":"2025-06-07T22:00:44.423660","end_time":null,"data":{},"created_at":"2025-06-07T22:00:44.150769","updated_at":"2025-06-07T22:00:44.167994"}...


In [4]:
user_id

1

In [8]:
while True:
    transcript = client.get_transcript(native_meeting_id=native_meeting_id,platform='google_meet')
    df = pd.DataFrame(transcript['segments'])#.sort_values(['created_at','start'])#.drop_duplicates('start',keep='last')
    clear_output()
    display(df.sort_values('absolute_start_time').tail(10))
    time.sleep(1)

,start,end,text,language,created_at,speaker,absolute_start_time,absolute_end_time
0,1.02,5.02,"1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 12.",ru,None,Dmitriy Grankin,2025-06-07T22:01:08.762962+00:00,2025-06-07T22:01:12.762962+00:00


KeyboardInterrupt: 

In [9]:
client.stop_bot(platform="google_meet",native_meeting_id=native_meeting_id)


DEBUG: Making DELETE request to http://localhost:18056/bots/google_meet/xdy-txjo-fyh
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': '9JT4p4Ms2WqGc951ZJtyu7oOltStI9onzWdFkEPC'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Sat, 07 Jun 2025 22:01:21 GMT, Sat, 07 Jun 2025 22:01:21 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '59', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Stop request accepted and is being processed."}...


{'message': 'Stop request accepted and is being processed.'}

In [69]:
user_id

16

In [70]:
running_bots = client.get_running_bots_status()


DEBUG: Making GET request to http://localhost:18056/bots/status
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'XZeo7ATOP3nkaxsyoH7aRJ9XdhnvQB6REXssPgaV'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Sat, 07 Jun 2025 13:56:07 GMT, Sat, 07 Jun 2025 13:56:07 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '415', 'content-type': 'application/json'}
DEBUG: Response content: {"running_bots":[{"container_id":"4a2183e92b2231893826b80161c1182ebc05aa1ba1f6b904113e7ee1c709c53a","container_name":"vexa-bot-13-3adbee41","platform":"google_meet","native_meeting_id":"xdy-txjo-fyh","status":"Up 34 seconds","created_at":"2025-06-07T13:55:33+00:00","labels":{"org.opencontainers.image.ref.name":"ubuntu","org.opencontainers.image.version":"22.04","vexa.user_id":"16"},"meeting_id_from_name":"13"}]}...


In [49]:
client.update_bot_config(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
    language='en',
    task = 'transcribe'
)


DEBUG: Making PUT request to http://localhost:18056/bots/google_meet/icj-aejh-tac/config
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'VLZbS0Vh3nNBNk8woojDWBG1r56cwP5PEIdPM74s'}
DEBUG: Params: None
DEBUG: JSON data: {'language': 'en', 'task': 'transcribe'}
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Wed, 04 Jun 2025 17:48:37 GMT, Wed, 04 Jun 2025 17:48:38 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '67', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Reconfiguration request accepted and sent to the bot."}...


{'message': 'Reconfiguration request accepted and sent to the bot.'}

In [ ]:
running_bots = client.get_running_bots_status()


DEBUG: Making GET request to http://localhost:8056/bots/status
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'yl9x4xUZdXnLZsRxfj6A7IQJEyf2czFpQBjV1e93'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Tue, 22 Apr 2025 13:58:29 GMT, Tue, 22 Apr 2025 13:58:29 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '1202', 'content-type': 'application/json'}
DEBUG: Response content: {"running_bots":[{"container_id":"e9c6def1ea1468b67b5bb8c010eff03febadf40e8b1727d6dadf8a72ddeb9c9a","container_name":"vexa-bot-11-4f4fd204","platform":"google_meet","native_meeting_id":"phu-przh-ove","status":"Up 30 minutes","created_at":"2025-04-22T13:27:49+00:00","labels":{"org.opencontainers.image.ref.name":"ubuntu","org.opencontainers.image.version":"22.04","vexa.user_id":"2"},"meeting_id_from_name":"11"},{"container_id":"433f2e1fd9776f6ddae1262b530b62b66b5a3bca16be9b25bfe1a7b10f3c3967","con...


In [10]:
meetings = client.get_meetings()


DEBUG: Making GET request to http://localhost:18056/meetings
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': '9JT4p4Ms2WqGc951ZJtyu7oOltStI9onzWdFkEPC'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Sat, 07 Jun 2025 22:01:26 GMT, Sat, 07 Jun 2025 22:01:26 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '496', 'content-type': 'application/json'}
DEBUG: Response content: {"meetings":[{"id":4,"user_id":4,"platform":"google_meet","native_meeting_id":"xdy-txjo-fyh","constructed_meeting_url":"https://meet.google.com/xdy-txjo-fyh","status":"completed","bot_container_id":"a06675e931d224b995711174211ab7e1e27a670ba0ee67c3b707dada795c98f5","start_time":"2025-06-07T22:00:44.423660","end_time":"2025-06-07T22:01:23.848721","data":{"languages":["ru"],"participants":["Dmitriy Grankin"]},"created_at":"2025-06-07T22:00:44.150769","updated_at":"2025-06-07T22:01:23.758023"}]}...


In [11]:
meetings

[{'id': 4,
  'user_id': 4,
  'platform': 'google_meet',
  'native_meeting_id': 'xdy-txjo-fyh',
  'constructed_meeting_url': 'https://meet.google.com/xdy-txjo-fyh',
  'status': 'completed',
  'bot_container_id': 'a06675e931d224b995711174211ab7e1e27a670ba0ee67c3b707dada795c98f5',
  'start_time': '2025-06-07T22:00:44.423660',
  'end_time': '2025-06-07T22:01:23.848721',
  'data': {'languages': ['ru'], 'participants': ['Dmitriy Grankin']},
  'created_at': '2025-06-07T22:00:44.150769',
  'updated_at': '2025-06-07T22:01:23.758023'}]

In [ ]:
meetings

[{'id': 3,
  'user_id': 3,
  'platform': 'google_meet',
  'native_meeting_id': 'xdy-txjo-fyh',
  'constructed_meeting_url': 'https://meet.google.com/xdy-txjo-fyh',
  'status': 'completed',
  'bot_container_id': 'decf57204ffd8bb66925335d27c48a3f621ad39958f5cd33b468da9a53bdd7f2',
  'start_time': '2025-06-07T12:07:27.806308',
  'end_time': '2025-06-07T12:08:15.643142',
  'data': {},
  'created_at': '2025-06-07T12:07:27.512952',
  'updated_at': '2025-06-07T12:08:15.633636'}]

In [ ]:
meetings

[{'id': 3,
  'user_id': 3,
  'platform': 'google_meet',
  'native_meeting_id': 'xdy-txjo-fyh',
  'constructed_meeting_url': 'https://meet.google.com/xdy-txjo-fyh',
  'status': 'completed',
  'bot_container_id': 'decf57204ffd8bb66925335d27c48a3f621ad39958f5cd33b468da9a53bdd7f2',
  'start_time': '2025-06-07T12:07:27.806308',
  'end_time': '2025-06-07T12:08:15.643142',
  'data': {},
  'created_at': '2025-06-07T12:07:27.512952',
  'updated_at': '2025-06-07T12:08:15.633636'}]

In [ ]:
meetings

[{'id': 1,
  'user_id': 1,
  'platform': 'google_meet',
  'native_meeting_id': 'xdy-txjo-fyh',
  'constructed_meeting_url': 'https://meet.google.com/xdy-txjo-fyh',
  'status': 'completed',
  'bot_container_id': '8d97700bc87f780736093e324ec24176f15a9466ecf4169e18c151b05642e01d',
  'start_time': '2025-06-07T11:54:09.676418',
  'end_time': '2025-06-07T11:56:44.720803',
  'data': {'name': 'Test Meeting'},
  'created_at': '2025-06-07T11:54:09.117279',
  'updated_at': '2025-06-07T11:56:44.710734'}]

In [7]:
client.update_meeting_data(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
    name="Test Meeting",
)


DEBUG: Making PATCH request to http://localhost:18056/meetings/google_meet/xdy-txjo-fyh
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'RhZQnJMyq8u4NfE1FrOOZWV9AFB1S2Xb37iDcJwT'}
DEBUG: Params: None
DEBUG: JSON data: {'data': {'name': 'Test Meeting'}}
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Sat, 07 Jun 2025 11:56:11 GMT, Sat, 07 Jun 2025 11:56:11 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '422', 'content-type': 'application/json'}
DEBUG: Response content: {"id":1,"user_id":1,"platform":"google_meet","native_meeting_id":"xdy-txjo-fyh","constructed_meeting_url":"https://meet.google.com/xdy-txjo-fyh","status":"active","bot_container_id":"8d97700bc87f780736093e324ec24176f15a9466ecf4169e18c151b05642e01d","start_time":"2025-06-07T11:54:09.676418","end_time":null,"data":{"name":"Test Meeting"},"created_at":"2025-06-07T11:54:09.117279","updated_at":"2025-06-07T11:56:11.794035"}...


{'id': 1,
 'user_id': 1,
 'platform': 'google_meet',
 'native_meeting_id': 'xdy-txjo-fyh',
 'constructed_meeting_url': 'https://meet.google.com/xdy-txjo-fyh',
 'status': 'active',
 'bot_container_id': '8d97700bc87f780736093e324ec24176f15a9466ecf4169e18c151b05642e01d',
 'start_time': '2025-06-07T11:54:09.676418',
 'end_time': None,
 'data': {'name': 'Test Meeting'},
 'created_at': '2025-06-07T11:54:09.117279',
 'updated_at': '2025-06-07T11:56:11.794035'}